In [ ]:
%load_ext autoreload
%autoreload 2
import glob
from itertools import chain

import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib_inline
import numpy as np
import orjson
import pandas as pd

matplotlib_inline.backend_inline.set_matplotlib_formats('svg')
plt.rcParams['figure.figsize'] = [14, 9]

pd.set_option('future.no_silent_downcasting', True)

In [ ]:
algo_order = [
    'age-pfifofast',
    'pfifo',
    'age-cake',
    'cake',
    'age-prio-fqcodel',
    'fqcodel',
    'age-prio-codel',
    'codel',
    'age-prio-fqpie',
    'fqpie',
    'age-prio-pie',
    'pie',
]

algonames = ['FIFO', 'CAKE', 'FQ-CoDel', 'CoDel', 'FQ-PIE', 'PIE']
algonames = chain.from_iterable((x,x) for x in algonames)
algotoname = dict(zip(algo_order, algonames))


In [ ]:
import re

testname_match_antler = re.compile(r'.*/([^/]+)/closedloop_([a-z\-]+)_procstat-(client|server).json')
def load_procstat(filename):
    with open(filename, 'rb') as f:
        df = pd.DataFrame(orjson.loads(f.read()))
    df['time'] = pd.to_datetime(df['time'], utc=True)
    df['time'] = df['time'] - df['time'].min()
    df['filename'] = filename
    date, algo, node = testname_match_antler.match(filename).groups()
    df['date'] = date
    df['algorithm'] = pd.Categorical([algo]*len(df), categories=algo_order, ordered=True)
    df['node'] = pd.Categorical([node]*len(df), categories=['client', 'server'], ordered=True)
    x = df['/proc/stat'].str.split()
    df['cpu_user'] = x.str[1].astype(int)
    df['cpu_system'] = x.str[3].astype(int)
    df['cpu_idle'] = x.str[4].astype(int)
    df['cpu_iowait'] = x.str[5].astype(int)
    df['cpu_irq'] = x.str[6].astype(int)
    df['cpu_softirq'] = x.str[7].astype(int)
    return df

pstats = pd.concat(load_procstat(filename) for filename in glob.glob('../tests/results/10gbps/*/*_procstat-*.json'))
pstats.info()

In [ ]:
def tot_cpu(series):
    return series.max() - series.min()

df = pstats.groupby(['date', 'node', 'algorithm'], observed=True).agg({c: (tot_cpu) for c in pstats.columns if c.startswith('cpu_')}).unstack('node') / (4*3600) / 100
# df = df[df.index.droplevel(1).str.contains('newpie') & (df.index.droplevel(1) != 'newpie1') | ~df.index.droplevel(0).str.contains('pie')]
df = df[df.index.droplevel(1).str.contains('newpie') & (df.index.droplevel(1) != 'newpie1') | ~df.index.droplevel(0).str.contains('pie') & (df.index.droplevel(1) != 'dead')]
df = df.groupby(['algorithm']).mean()
df

In [ ]:
def name(x):
    if x.startswith('age-'):
        return 'Size-b. ' + algotoname[x]
    return algotoname[x]

cols = [
    ('cpu_user', 'client'),
    ('cpu_system', 'client'),
    ('cpu_iowait', 'client'),
    ('cpu_irq', 'client'),
    ('cpu_softirq', 'client'),
]

print('\\hline')
print('{Algorithm} & {User} & {System} & {IO Wait} & {IRQ} & {Soft IRQ} \\\\')
print('\\hline \\hline')

for row in df[cols].itertuples():
    print(
        f'{name(row.Index):<20} & '
        f'{row[1]:.3g} & {row[2]:.3g} & {row[3]:.3g} & {row[4]:.3g} & {row[5]:.3g} '
        '\\\\',
    )
    if not row.Index.startswith('age-'):
        print('\\hline')